In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import stan # python3 -m pip install pystan
import nest_asyncio
nest_asyncio.apply()
import warnings

In [25]:
df = pd.read_csv("Number_Pivoted.csv")
df.head()

,Year,[0],[1-4],[10-14],[15-19],[20-24],[25-29],[30-34],[35-39],[40-44],...,[5-9],[50-54],[55-59],[60-64],[65-69],[70-74],[75-79],[80-84],[85+],[All]
0,2000,163.0,567.0,936.0,5176.0,5264.0,3679.0,3153.0,3549.0,3392.0,...,751.0,2413.0,1890.0,1581.0,1414.0,1583.0,1724.0,1390.0,1276.0,42866.0
1,2001,140.0,564.0,899.0,5155.0,5465.0,3675.0,3196.0,3522.0,3549.0,...,669.0,2557.0,1915.0,1518.0,1393.0,1571.0,1792.0,1459.0,1209.0,43328.0
2,2002,122.0,541.0,900.0,5549.0,5760.0,3689.0,3343.0,3441.0,3616.0,...,636.0,2700.0,2005.0,1755.0,1456.0,1615.0,1734.0,1506.0,1264.0,44937.0
3,2003,146.0,510.0,929.0,5265.0,5678.0,3649.0,3195.0,3293.0,3696.0,...,608.0,2798.0,2252.0,1741.0,1430.0,1533.0,1655.0,1521.0,1337.0,44555.0
4,2004,140.0,524.0,941.0,5180.0,5726.0,3825.0,3162.0,3195.0,3470.0,...,601.0,2867.0,2287.0,1789.0,1549.0,1414.0,1650.0,1482.0,1242.0,44505.0


In [48]:
data = dict(N=len(df), x=df["Year"].to_numpy(), y=df["[All]"].to_numpy())

In [54]:
file_path = "stan/linear.stan"

# Read the Stan code from the file
with open(file_path, "r") as file:
    stan_code = file.read()
print(stan_code)

warnings.filterwarnings("ignore")

posterior = stan.build(stan_code, data=data)

data {
  int<lower=0> N;
  vector[N] x;
  vector[N] y;
}
parameters {
  real alpha;
  real beta;
  real<lower=0> sigma;
}
model {
  // priors
  alpha ~ normal(42000, 1000);
  beta ~ normal(0, 100);
  sigma ~ cauchy(0, 2.5); 
  
  y ~ normal(alpha + beta * x, sigma);
}
Building...

In file included from /home/mantyke1/.cache/httpstan/4.6.1/models/sww5sjly/model_sww5sjly.cpp:2:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/model/model_header.hpp:4:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/math.hpp:19:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/math/rev.hpp:8:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/math/rev/core.hpp:28:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/math/rev/core/operator_addition.hpp:6:
/opt/software/lib/python3.10/site-packages/httpstan/include/stan/math/prim/err/check_matching_dims.hpp:56:23: warning: comparison of integers of different signs: 'int' and 'std::vector::size_type' (aka 'unsigned long') [-Wsign-compare]
    for (int i = 0; i < y1_d.size(); i++) {
                    ~ ^ ~~~~~~~~~~~
In file included from /home/mantyke1


Building: 15.4s, done.Messages from stanc:
Warning in '/tmp/httpstan_pfb_u_73/model_sww5sjly.stan', line 14, column 19: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pfb_u_73/model_sww5sjly.stan', line 13, column 24: Argument
    1000 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pfb_u_73/model_sww5sjly.stan', line 13, column 17: Argument
    42000 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).


In [55]:
fit = posterior.sample(num_chains=4, num_samples=1000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 1.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.18 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.16 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!


In [56]:
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,alpha,beta,sigma
draws,,,,,,,,,,
0,-201.760861,0.898534,0.443383,3.0,7.0,0.0,202.934676,41689.736548,0.249459,4324.382443
1,-201.641126,0.679302,0.463953,2.0,7.0,0.0,206.873031,40599.516922,0.533775,4163.797258
2,-200.551231,0.908793,0.459487,3.0,7.0,0.0,203.103560,41660.385274,-0.221690,4619.709548
3,-199.644216,0.994477,0.425229,3.0,7.0,0.0,199.893531,42128.517594,-1.086439,3525.336162
4,-201.662443,0.734022,0.443383,3.0,7.0,0.0,205.658725,40091.489169,0.515603,3987.226961
...,...,...,...,...,...,...,...,...,...,...
3995,-200.725141,0.969504,0.425229,3.0,11.0,0.0,203.909732,42461.163387,-0.631715,3302.508921
3996,-202.447210,0.896879,0.443383,3.0,7.0,0.0,206.119997,40168.178423,-0.611843,4191.904180
3997,-199.924191,0.975585,0.463953,3.0,7.0,0.0,201.832805,42739.198675,-0.995222,3518.078709
